### Function to test a bedrock model

In [37]:
# Test a bedrock model given a user message and an inference configuration 

import traceback
import boto3
from botocore.exceptions import ClientError

def test_bedrock_model(
    bedrock_client,
    model_id,
    user_message,
    inferenceConfig={"maxTokens": 512, "temperature": 0.5, "topP": 0.9}
):
    # Start a conversation with the user message.
    conversation = [{
        "role": "user",
        "content": [{"text": user_message}],
    }]
    
    try:
        # Send the message to the model, using a basic inference configuration.
        response = bedrock_client.converse(
            modelId=model_id,
            messages=conversation,
            inferenceConfig=inferenceConfig
        )
    
        # Extract and print the response text.
        response_text = response["output"]["message"]["content"][0]["text"]
        return {'ok': True, 'answer': response_text}
    
    except (ClientError, Exception) as e:
        print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
        return {'ok': False, 'error': traceback.format_exc()}

### Get models

In [40]:
import boto3
import pandas as pd
# pd.set_option('display.max_rows', 100)

bedrock_client = boto3.client(service_name="bedrock", region_name="us-east-1")

response = bedrock_client.list_foundation_models()
models = response["modelSummaries"]
models = pd.DataFrame(models)

display(models.iloc[:, [1, 2, 3, 4, 8, 9]].head(5))

,modelId,modelName,providerName,inputModalities,inferenceTypesSupported,modelLifecycle
0,amazon.titan-tg1-large,Titan Text Large,Amazon,[TEXT],[ON_DEMAND],{'status': 'ACTIVE'}
1,amazon.titan-image-generator-v1:0,Titan Image Generator G1,Amazon,"[TEXT, IMAGE]",[PROVISIONED],{'status': 'ACTIVE'}
2,amazon.titan-image-generator-v1,Titan Image Generator G1,Amazon,"[TEXT, IMAGE]",[ON_DEMAND],{'status': 'ACTIVE'}
3,amazon.titan-image-generator-v2:0,Titan Image Generator G1 v2,Amazon,"[TEXT, IMAGE]","[PROVISIONED, ON_DEMAND]",{'status': 'ACTIVE'}
4,amazon.titan-text-premier-v1:0,Titan Text G1 - Premier,Amazon,[TEXT],[ON_DEMAND],{'status': 'ACTIVE'}


#### Get all models with access provided

AWS provides access to specific model families, each containing different versions of similar models.

The code below lists the model available given a list of model families with access already provided.

In [42]:
model_families_with_access_provided = [
    'Titan Text G1 - Express',
    'Titan Text Embeddings V2',
    'Claude 3.5 Sonnet',
    'Claude 3 Sonnet',
    'Claude 3 Haiku',
    'Llama 3.2 90B Vision Instruct',
    'Llama 3 8B Instruct',
    'Llama 3 70B Instruct',
    'Mistral Large (24.02)'
]

accessible_models = models[models['modelName'].isin(models_with_access_provided)]

print('Total model families accessible:', len(models_with_access_provided))
print('Total models accessible:', len(accessible_models))
print()
display(accessible_models.iloc[:, [1, 2, 3]])

Total model families accessible: 9
Total models accessible: 14



,modelId,modelName,providerName
16,amazon.titan-text-express-v1:0:8k,Titan Text G1 - Express,Amazon
17,amazon.titan-text-express-v1,Titan Text G1 - Express,Amazon
20,amazon.titan-embed-text-v2:0:8k,Titan Text Embeddings V2,Amazon
21,amazon.titan-embed-text-v2:0,Titan Text Embeddings V2,Amazon
44,anthropic.claude-3-sonnet-20240229-v1:0:28k,Claude 3 Sonnet,Anthropic
45,anthropic.claude-3-sonnet-20240229-v1:0:200k,Claude 3 Sonnet,Anthropic
46,anthropic.claude-3-sonnet-20240229-v1:0,Claude 3 Sonnet,Anthropic
47,anthropic.claude-3-haiku-20240307-v1:0:48k,Claude 3 Haiku,Anthropic
48,anthropic.claude-3-haiku-20240307-v1:0:200k,Claude 3 Haiku,Anthropic
49,anthropic.claude-3-haiku-20240307-v1:0,Claude 3 Haiku,Anthropic


### Test multiple models

- Use the same prompt
- Use the same combination of parameters

In [75]:
bedrock_client = boto3.client("bedrock-runtime", region_name="us-east-1")

user_message = "Describe the purpose of a 'hello world' program in one line."
inferenceConfig = {
    "maxTokens": 512,
    "temperature": 0.5,
    "topP": 0.9
}

results = []
for _, row in accessible_models.iterrows():
    model_id = row['modelId']
    
    result = test_bedrock_model(
        bedrock_client,
        model_id,
        user_message,
        inferenceConfig
    )
    result = {**row.iloc[[1, 2, 3]].to_dict(), **result}
    results.append(result)

results = pd.DataFrame(results)

ERROR: Can't invoke 'amazon.titan-text-express-v1:0:8k'. Reason: An error occurred (ResourceNotFoundException) when calling the Converse operation: Model not found.
ERROR: Can't invoke 'amazon.titan-embed-text-v2:0:8k'. Reason: An error occurred (ResourceNotFoundException) when calling the Converse operation: Model not found.
ERROR: Can't invoke 'amazon.titan-embed-text-v2:0'. Reason: An error occurred (ValidationException) when calling the Converse operation: This action doesn't support the model that you provided. Try again with a supported text or chat model.
ERROR: Can't invoke 'anthropic.claude-3-sonnet-20240229-v1:0:28k'. Reason: An error occurred (ResourceNotFoundException) when calling the Converse operation: Model not found.
ERROR: Can't invoke 'anthropic.claude-3-sonnet-20240229-v1:0:200k'. Reason: An error occurred (ResourceNotFoundException) when calling the Converse operation: Model not found.
ERROR: Can't invoke 'anthropic.claude-3-haiku-20240307-v1:0:48k'. Reason: An err

### Evaluate Result

In [77]:
print(f'Success: {results['ok'].sum()}/{len(results)}')

Success: 7/14


In [81]:
pd.set_option('display.max_colwidth', None)

results[results['ok']]

,modelId,modelName,providerName,ok,error,answer
1,amazon.titan-text-express-v1,Titan Text G1 - Express,Amazon,True,NaN,\n\nA 'hello world' program is a simple computer program that outputs the phrase 'hello world' to the screen. It is often used as an introductory example to teach programming concepts and as a benchmark to compare the performance of different programming languages or systems.
6,anthropic.claude-3-sonnet-20240229-v1:0,Claude 3 Sonnet,Anthropic,True,NaN,The purpose of a 'hello world' program is to serve as a simple introduction to programming in a particular language or environment.
9,anthropic.claude-3-haiku-20240307-v1:0,Claude 3 Haiku,Anthropic,True,NaN,"A 'hello world' program is a simple, introductory program that serves as a starting point for learning a new programming language or testing a development environment."
10,anthropic.claude-3-5-sonnet-20240620-v1:0,Claude 3.5 Sonnet,Anthropic,True,NaN,"A ""Hello World"" program serves as a simple introduction to a programming language or environment by demonstrating the basic syntax for outputting text."
11,meta.llama3-8b-instruct-v1:0,Llama 3 8B Instruct,Meta,True,NaN,"\n\nThe purpose of a ""Hello World"" program is to serve as a simple, introductory example that demonstrates the basic syntax and functionality of a programming language or environment, typically used as a starting point for beginners to learn and experiment with programming."
12,meta.llama3-70b-instruct-v1:0,Llama 3 70B Instruct,Meta,True,NaN,"\n\nA ""hello world"" program is a traditional introductory exercise in programming that serves as a sanity check to ensure a development environment is set up correctly and to verify that a programmer can successfully compile and run a simple program."
13,mistral.mistral-large-2402-v1:0,Mistral Large (24.02),Mistral AI,True,NaN,"The purpose of a 'hello world' program is to provide a simple introduction to a programming language, demonstrating its basic syntax and giving beginners a tangible result - typically, displaying the text ""Hello, World!"" on the screen."
